<a href="https://colab.research.google.com/github/yshalsager/OpenAI_Whisper_ytdlp/blob/master/OpenAI_Whisper_ar_ytdlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# تفريغ الصوتيات باستخدم OpenAI Whisper

هذا الملف مبني على [هذا الملف](https://colab.research.google.com/gist/Kazuki-tam/04e85708e4fd1c4b8af180d317977f4d/whisper-mock-en.ipynb)


## 📖 كيفية الاستخدام
1. Run "Setting up".
2. Open the folder icon from the left sidebar.
3. Upload audio files into the `content`.
4. Input the audio file name into `fileName`.
5. Select output language.
5. Run "Transcription".

In [ ]:
#@title اﻹعداد
# Install packages
!pip install git+https://github.com/openai/whisper.git
!pip install yt-dlp
!wget https://raw.githubusercontent.com/jianfch/stable-ts/dcf3c55e6655ddee5694ad442c93d8f49fe476de/stable_whisper.py -O stable_whisper.py

from pathlib import Path

# Add folders
download_folder = Path("download")
if not download_folder.exists():
  download_folder.mkdir()
output_folder = Path("output")
if not output_folder.exists():
  output_folder.mkdir()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-u23cpmug
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-u23cpmug
  Resolved https://github.com/openai/whisper.git to commit 5c1a8c10e762bf9c29fcf6b3e40f17bc8ab09864
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230124-py3-none-any.whl size=1179333 sha256=50bf43e6434227db51df5ca74317c171ccb3b135d6426894f6c5e3df6de60943
  Stored in directory: /tmp/pip-ephem-wheel-cache-xv4k43db/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built opena

In [ ]:
#@title Transcription
import whisper
import yt_dlp

from pathlib import Path
from google.colab import files
from stable_whisper import modify_model, results_to_sentence_srt

youtube_urls = ""#@param {type:"string"}
playlist_start = 1 #@param {type:"integer"}
playlist_end = 9999 #@param {type:"integer"}

ydl_opts = {
    'format': 'm4a/bestaudio/best',
    'postprocessors': [{  # Extract audio using ffmpeg
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'm4a',
    }],
    "playliststart": int(playlist_start),
    "playlistend": int(playlist_end),
    "outtmpl": f"{str(download_folder)}/%(playlist_index)04d-%(title)s-%(id)s.%(ext)s"
}

for youtube_url in youtube_urls.split():
  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      ydl.download(youtube_url)

language = "ar"#@param ["en", "ar", "Afrikaans", "Albanian", "Amharic", "Arabic", "Armenian", "Assamese", "Azerbaijani", "Bashkir", "Basque", "Belarusian", "Bengali", "Bosnian", "Breton", "Bulgarian", "Burmese", "Castilian", "Catalan", "Chinese", "Croatian", "Czech", "Danish", "Dutch", "English", "Estonian", "Faroese", "Finnish", "Flemish", "French", "Galician", "Georgian", "German", "Greek", "Gujarati", "Haitian", "Haitian Creole", "Hausa", "Hawaiian", "Hebrew", "Hindi", "Hungarian", "Icelandic", "Indonesian", "Italian", "Japanese", "Javanese", "Kannada", "Kazakh", "Khmer", "Korean", "Lao", "Latin", "Latvian", "Letzeburgesch", "Lingala", "Lithuanian", "Luxembourgish", "Macedonian", "Malagasy", "Malay", "Malayalam", "Maltese", "Maori", "Marathi", "Moldavian", "Moldovan", "Mongolian", "Myanmar", "Nepali", "Norwegian", "Nynorsk", "Occitan", "Panjabi", "Pashto", "Persian", "Polish", "Portuguese", "Punjabi", "Pushto", "Romanian", "Russian", "Sanskrit", "Serbian", "Shona", "Sindhi", "Sinhala", "Sinhalese", "Slovak", "Slovenian", "Somali", "Spanish", "Sundanese", "Swahili", "Swedish", "Tagalog", "Tajik", "Tamil", "Tatar", "Telugu", "Thai", "Tibetan", "Turkish", "Turkmen", "Ukrainian", "Urdu", "Uzbek", "Valencian", "Vietnamese", "Welsh", "Yiddish", "Yoruba", "af", "am", "as", "az", "ba", "be", "bg", "bn", "bo", "br", "bs", "ca", "cs", "cy", "da", "de", "el", "es", "et", "eu", "fa", "fi", "fo", "fr", "gl", "gu", "ha", "haw", "hi", "hr", "ht", "hu", "hy", "id", "is", "it", "iw", "ja", "jw", "ka", "kk", "km", "kn", "ko", "la", "lb", "ln", "lo", "lt", "lv", "mg", "mi", "mk", "ml", "mn", "mr", "ms", "mt", "my", "ne", "nl", "nn", "no", "oc", "pa", "pl", "ps", "pt", "ro", "ru", "sa", "sd", "si", "sk", "sl", "sn", "so", "sq", "sr", "su", "sv", "sw", "ta", "te", "tg", "th", "tk", "tl", "tr", "tt", "uk", "ur", "uz", "vi", "yi", "yo", "zh"]
model = "large"#@param ["large", "medium", "base", "small", "tiny"]
model = whisper.load_model(model)
# jianfch/stable-ts
modify_model(model)

for audio_file in sorted(download_folder.glob("*.m4a")):
  print(f"Transcription of {audio_file} will start!")
  text_file = Path(f"{output_folder}/{audio_file.stem}.txt")
  subtitle_file = Path(f"{output_folder}/{audio_file.stem}.srt")

  result = model.transcribe(str(audio_file))
  # save TXT
  with open(str(text_file), "w", encoding="utf-8") as txt:
    for segment in result["segments"]:
        txt.write(segment['text'].strip() + '\n')
  files.download(str(text_file))
  # save SRT
  #with open(f"{download_folder}/{audio_file.name}.srt", "w", encoding="utf-8") as srt:
  #    whisper.write_srt(result["segments"], file=srt)
  results_to_sentence_srt(result, str(subtitle_file))
  files.download(str(subtitle_file))
  ## Write into a text file
  #Path(f"{download_folder}/{file_name.name}.txt").write_text(result["text"])
  audio_file.unlink()
print("Done!")

[youtube] Extracting URL: iQyL7hP2Drk
[youtube] iQyL7hP2Drk: Downloading webpage
[youtube] iQyL7hP2Drk: Downloading android player API JSON
[info] iQyL7hP2Drk: Downloading 1 format(s): 140
[download] Destination: download/NA-ألف باء العلوم ｜ درس النساء ｜ فقه الإخلاص 07-iQyL7hP2Drk.m4a
[download] 100% of  154.63MiB in 00:00:14 at 10.93MiB/s  
[FixupM4a] Correcting container of "download/NA-ألف باء العلوم ｜ درس النساء ｜ فقه الإخلاص 07-iQyL7hP2Drk.m4a"
[ExtractAudio] Not converting audio download/NA-ألف باء العلوم ｜ درس النساء ｜ فقه الإخلاص 07-iQyL7hP2Drk.m4a; file is already in target format m4a


100%|█████████████████████████████████████| 2.87G/2.87G [01:03<00:00, 48.5MiB/s]


Transcription of download/NA-ألف باء العلوم ｜ درس النساء ｜ فقه الإخلاص 07-iQyL7hP2Drk.m4a will start!
Detected language: arabic


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved: /content/output/NA-ألف باء العلوم ｜ درس النساء ｜ فقه الإخلاص 07-iQyL7hP2Drk.srt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Done!
